# Errors - Monte Carlo simulations 

## In this project, I will show how to extract errors on best-fit parameters estimated from the curve fitting techniques introduced in  __[curve_fitting](https://github.com/fakahil/Projects/blob/master/curve_fitting.ipynb)__. I will use the same example (the limb data) for illustration.

In [1]:
## importing the libraries
import numpy as np
import math as m
import scipy
import matplotlib.pyplot as plt
from scipy import special
from scipy.optimize import curve_fit

In [2]:
def Erfc(x,sigma):

 y = special.erfc(x/(sigma*np.sqrt(2)))
 return y

## Introducing the model to be used later for the fitting

def SL_fit(x,w1,w2,w3,s1,s2,s3):

  f = 0.5*(w1*Erfc(x,s1)+w2*Erfc(x,s2)+w3*Erfc(x,s3)+ (1-w1-w2-w3))
  return f

### From now on I will use __[scipy.optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)__  for the fitting problem

In [3]:
## Jacobian matrix
def DF(x,w1,w2,w3,s1,s2,s3):
 dfw1 = 0.5*(Erfc(x,s1)-1)
 dfw2 = 0.5*(Erfc(x,s2)-1)
 dfw3 =  0.5*(Erfc(x,s3)-1)
 dfs1 = w1*x*np.exp(-x**2/(2*s1**2))/ (np.sqrt(2*np.pi)*s1**2)
 dfs2 = w2*x*np.exp(-x**2/(2*s2**2))/ (np.sqrt(2*np.pi)*s2**2)
 dfs3 = w3*x*np.exp(-x**2/(2*s3**2))/ (np.sqrt(2*np.pi)*s3**2)
 return np.transpose(np.array([dfw1,dfw2,dfw3,dfs1,dfs2,dfs3]))


In [4]:
## Loading the data:
path = "/home/fatima/Desktop/project_3/"
file = np.loadtxt(path+'limb_profile_av_norm_shifted')
x = file[:,0]
y = file[:,1]

In [5]:
ind = np.where(x>=0)
x = x[ind]
y = y[ind]
weights = np.sqrt(np.abs(y)) ## Poisson weighting

In [6]:
p0=[0.3, 0.3, 0.2, 1, 2, 3] ## initial guess best-fit parameters
popt, pcov = curve_fit(SL_fit,x,y,p0,method='lm',sigma=weights,jac=DF,ftol=1e-8,xtol=1e-8,maxfev=5000)
chi_sq_w = np.sum((1/weights**2)*(SL_fit(x,*popt)-y)**2)
red_chi_sq = chi_sq_w/(len(y)-len(popt))
print popt # to print the best-fit parameters

[ 0.52750103  0.28882569  0.10191755  0.25905336  0.76540583  2.83343005]


### In the lines above, popt is the array of best-fit parameters, while pcov is the covariance matrix, the diagonal of which can give an estimate of the errors by typing the following:

In [7]:
perr = np.sqrt(np.diag(pcov))
print perr ## to print the error

[ 0.00349415  0.0028457   0.00119396  0.00135734  0.00715294  0.04738134]


### One can also use another method to compute the errors on the best-fit parameters. The so called "Monte Carlo" method. 

In [8]:
### The "true" solution is assumed to be that returned by curve_fit, popt
w10, w20, w30, s10, s20, s30 = popt
w40 = 1-w10-w20-w30

In [9]:
### we will make a copy for our data set so we don't alter the original data set
xr = x.copy()
yr = y.copy()
ery = weights.copy()

In [12]:
N = len(y)
trials = 1000 ## number of synthetic data sets
omegas1, omegas2, omegas3,omegas4,sigmas1,sigmas2, sigmas3 = [],[],[],[],[],[],[] #defining the arrays of best-fit parameters corresponding to the number of trials "trials"
chi_s = [] ## the array of dimension "trials" of chi-square values 

### Now generating synthetic data sets using  __[numpy.random.randint](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randint.html)__. 

In [13]:
from numpy.random import normal, randint

In [14]:
for i in range(trials):       
 indx = randint(0, N, N)    
 xr[:] = x[indx]
 yr[:] = y[indx]
 ery[:] = weights[indx]  
 ok = (xr != xr[0]).any()

 if (not ok):
   print "All elements are the same. Invalid sample."
   print xr, yr
 else:
    
    ## now preforming the fit using curve_fit on the synthetic data sets and saving results to popti
   popti, pcovi = curve_fit(SL_fit,xr,yr,p0,method='lm',sigma=ery,ftol=1e-8,xtol=1e-8,maxfev=5000)
   chi_sq_wi = np.sum((1/ery**2)*(SL_fit(xr,*popti)-yr)**2) ## chi_squared values
   red_chi_sqi = chi_sq_wi/(len(yr)-len(popti)) ## reduced chi_squared values
   om1, om2,om3,sig1,sig2,sig3 = popti
   om4 = 1-om1-om2-om3
## saving best-fit parameters and reduced chi_sq values
   omegas1.append(om1)
   omegas2.append(om2)
   omegas3.append(om3)
   omegas4.append(om4)
   sigmas1.append(sig1)
   sigmas2.append(sig2)
   sigmas3.append(sig3)
   chi_s.append(red_chi_sqi)
   print popti ##to print the output for each trial 

[ 0.53512958  0.28375438  0.10005328  0.26299095  0.78166362  2.90605193]
[ 0.53032147  0.28645877  0.10185095  0.26016222  0.76902722  2.85920675]
[ 0.53400267  0.28610429  0.09898945  0.26223567  0.7851513   2.93169158]
[ 0.52897877  0.28677786  0.10208443  0.25907955  0.76673213  2.80283091]
[ 0.53293181  0.28516127  0.10089723  0.26125326  0.77211599  2.89760184]
[ 0.52192964  0.29130126  0.1045936   0.25872931  0.74982372  2.77019247]
[ 0.52243348  0.291395    0.10407572  0.25669087  0.75293616  2.78355131]
[ 0.53045504  0.28595796  0.10174111  0.25907911  0.77064005  2.83012909]
[ 0.53434239  0.28284056  0.10132352  0.2641633   0.77314394  2.8591918 ]
[ 0.52912333  0.28731075  0.10216531  0.25963015  0.7658729   2.85451291]
[ 0.52363511  0.29102025  0.10325503  0.25667591  0.75721632  2.79243267]
[ 0.53826874  0.28177118  0.09902066  0.26263249  0.7884794   2.92831894]
[ 0.52601769  0.28994797  0.10250782  0.25808039  0.75953339  2.8411467 ]
[ 0.54328363  0.27733233  0.09873152  

/usr/local/lib/python2.7/dist-packages/scipy/optimize/minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


[ 0.52309093  0.29117606  0.10346057  0.25772544  0.75400897  2.77810357]
[ 0.51567906  0.29787626  0.10393295  0.25443144  0.74486343  2.75461301]
[ 0.53132837  0.28773146  0.09982852  0.25972908  0.77779875  2.90473103]
[ 0.53841606  0.28170713  0.09895059  0.26139372  0.78944134  2.93185574]
[ 0.52039116  0.2932246   0.10398732  0.25919371  0.74655729  2.76392622]
[ 0.52957371  0.28852727  0.10056215  0.26219427  0.76999048  2.88503903]
[ 0.52273239  0.29245521  0.10278328  0.25727943  0.76117348  2.80014237]
[ 0.52715708  0.28883894  0.10225559  0.26012588  0.76410136  2.83285668]
[ 0.53495779  0.28084996  0.10219897  0.26161311  0.77028151  2.80858654]
[ 0.52606435  0.28990856  0.10224222  0.2576801   0.76154749  2.82891662]
[ 0.52681549  0.29065897  0.10093366  0.25912282  0.76637057  2.86300246]
[ 0.53772081  0.28077249  0.10012984  0.26449465  0.78251891  2.88498384]
[ 0.52922879  0.28753797  0.10163368  0.26127877  0.77156253  2.84484319]
[ 0.52574884  0.28938414  0.1028707   

[ 0.52534545  0.29168525  0.10138548  0.25795348  0.76156626  2.86045527]
[ 0.52955292  0.28672486  0.10179613  0.26071343  0.7679081   2.8208997 ]
[ 0.52230919  0.29382257  0.1021837   0.25686407  0.75976859  2.83273564]
[ 0.52552407  0.2890179   0.103208    0.26007043  0.75555292  2.7788171 ]
[ 0.54111154  0.27720732  0.10005323  0.26269641  0.78755747  2.86418487]
[ 0.52804408  0.28765624  0.10211577  0.25740396  0.76452454  2.79651758]
[ 0.5289871   0.28902418  0.10059023  0.25851095  0.77201161  2.87644254]
[ 0.52663309  0.28923609  0.10221509  0.25948627  0.76451595  2.81763284]
[ 0.52906817  0.28788419  0.10142157  0.2598651   0.76613497  2.85341603]
[ 0.5304698   0.28548228  0.10235511  0.26033186  0.76613217  2.8310651 ]
[ 0.52669779  0.28971721  0.1019798   0.25783498  0.76390789  2.84352691]
[ 0.52498367  0.29052687  0.10286664  0.2583636   0.75910823  2.83136713]
[ 0.52670973  0.28930168  0.10205513  0.25746465  0.76292009  2.81810207]
[ 0.52008473  0.29385039  0.10407137  

[ 0.53941039  0.27971061  0.09964231  0.26461224  0.78836993  2.89870629]
[ 0.5288083   0.28786273  0.10163341  0.25969277  0.76787934  2.83684199]
[ 0.53901322  0.28006696  0.100101    0.26504425  0.78436762  2.92876897]
[ 0.52205926  0.29229501  0.10373414  0.25849942  0.75107997  2.79876394]
[ 0.52748768  0.28875416  0.10182357  0.25931938  0.76231321  2.82331019]
[ 0.52097734  0.2929448   0.10370355  0.2577157   0.75226443  2.76510941]
[ 0.52954049  0.28527801  0.10317559  0.25991368  0.76314605  2.79910007]
[ 0.53271228  0.28350256  0.10165742  0.26168216  0.76820309  2.81609558]
[ 0.53574737  0.28211614  0.10059     0.26340719  0.77908984  2.86326048]
[ 0.52570092  0.28854478  0.10311059  0.25984441  0.75934783  2.75475234]
[ 0.53837877  0.28257219  0.09829043  0.26237332  0.7925942   2.95534067]
[ 0.28919411  0.52418815  0.10423208  0.75585428  0.25815592  2.75505719]
[ 0.53763788  0.28089088  0.10026881  0.2633678   0.78117194  2.89726413]
[ 0.5342562   0.28348156  0.10072833  

[ 0.53363999  0.28567805  0.09968146  0.26057457  0.78034287  2.91754939]
[ 0.53374005  0.28536674  0.09976553  0.26129608  0.78222154  2.91163463]
[ 0.52696336  0.28869509  0.10255421  0.25603876  0.76160191  2.82038645]
[ 0.53281664  0.28335205  0.10199193  0.26157493  0.77042308  2.82235616]
[ 0.53539003  0.2839898   0.09968546  0.26142961  0.78427801  2.92379111]
[ 0.52662863  0.28851124  0.10283901  0.25856998  0.76296253  2.80208539]
[ 0.52896201  0.28854404  0.10091441  0.25934401  0.77034198  2.86208911]
[ 0.52221045  0.29301854  0.10256932  0.25637282  0.75644266  2.79005082]
[ 0.53002431  0.28837561  0.1005117   0.25889567  0.7707603   2.90035961]
[ 0.51945055  0.29680444  0.1020966   0.25632943  0.75447298  2.84145951]
[ 0.52664541  0.28794765  0.10321634  0.25891059  0.75952905  2.78467255]
[ 0.51856633  0.29420802  0.10446319  0.25455922  0.7469182   2.72907083]
[ 0.51911058  0.29536347  0.10332779  0.25551014  0.75304077  2.78313257]
[ 0.5156846   0.2981432   0.10421188  

[ 0.52093375  0.29384147  0.1032427   0.25733951  0.75342969  2.80272545]
[ 0.52416742  0.29031576  0.10335799  0.25935911  0.75643229  2.78510074]
[ 0.52615365  0.2908939   0.10146084  0.25832759  0.7640451   2.85509339]
[ 0.52478222  0.29006796  0.10305307  0.25741472  0.75686634  2.79340776]
[ 0.53605042  0.28185537  0.10055837  0.26373432  0.77901447  2.87046691]
[ 0.5129598   0.30030915  0.10419796  0.25381992  0.74227936  2.74913567]
[ 0.53303892  0.2837618   0.10153875  0.26077043  0.77035788  2.84755444]
[ 0.533692    0.28346068  0.10146805  0.26221212  0.7731479   2.86959559]
[ 0.51705784  0.29914507  0.10216578  0.25480503  0.75389194  2.83997393]
[ 0.52067411  0.29422016  0.10304246  0.25591774  0.7555216   2.80150417]
[ 0.53577939  0.28317833  0.09987589  0.26312833  0.78097928  2.90208665]
[ 0.28815893  0.52715432  0.10271561  0.76061359  0.25824735  2.80759471]
[ 0.53551239  0.28174285  0.10098147  0.26381156  0.77656363  2.84210457]
[ 0.52918805  0.28759343  0.10180158  

[ 0.5333748   0.28483746  0.10049711  0.26174244  0.77724009  2.88397749]
[ 0.52611917  0.28919609  0.10238364  0.25793065  0.76568353  2.78944762]
[ 0.52591238  0.29082257  0.10166898  0.25849271  0.76211802  2.85376416]
[ 0.52982711  0.28782155  0.10085034  0.25987206  0.77388339  2.86309013]
[ 0.53325009  0.28508838  0.10038462  0.25869917  0.77700404  2.88937434]
[ 0.53900044  0.28185353  0.09859771  0.26354715  0.79104139  2.96458224]
[ 0.54528643  0.27452911  0.09907908  0.26611255  0.7954479   2.91375904]
[ 0.52697822  0.28974684  0.10149351  0.25939803  0.76737407  2.8355175 ]
[  5.96748593e-02   6.53608950e-01   1.94624526e-01   1.96872500e-04
   3.67427650e-01   1.67450269e+00]
[ 0.52873544  0.28939173  0.10035891  0.25958945  0.77160722  2.87180451]
[ 0.52616219  0.28849535  0.10295518  0.2591736   0.76130699  2.77942681]
[ 0.53086949  0.28547336  0.10202691  0.26022548  0.76729131  2.84277569]
[ 0.52363932  0.29292629  0.10202438  0.25813399  0.76306115  2.85552482]
[ 0.535

[ 0.525794    0.28935023  0.10253822  0.25834473  0.76111513  2.78592245]
[ 0.5275403   0.28856215  0.10214223  0.25968302  0.76258331  2.82931643]
[ 0.53101766  0.28668067  0.10098738  0.26071173  0.77198947  2.87644077]
[ 0.52555183  0.28936419  0.10314909  0.25892968  0.75629062  2.81051535]
[ 0.51902198  0.29512397  0.10344962  0.25548321  0.75291017  2.77019874]
[ 0.522077    0.29397901  0.10228465  0.25565493  0.75793419  2.83553501]
[ 0.51802615  0.29616722  0.10343554  0.25487687  0.74868451  2.77765552]
[ 0.52112455  0.29400635  0.10285366  0.25596636  0.75816422  2.80117201]
[ 0.53159092  0.28680491  0.1005495   0.25803306  0.77543856  2.90160245]
[ 0.5237125   0.29142029  0.10326917  0.25731164  0.75675942  2.83132154]
[ 0.52294711  0.29304408  0.10247247  0.25702875  0.75996735  2.84401706]
[ 0.54034386  0.28024009  0.09887339  0.26511105  0.79270066  2.95590954]
[ 0.52711755  0.28862508  0.10204234  0.25890269  0.7643801   2.80158692]
[ 0.53065702  0.28597833  0.10170158  

[ 0.52410763  0.29068177  0.10335417  0.25626843  0.7585109   2.80505373]
[ 0.53474     0.28271961  0.10110922  0.26309473  0.77485297  2.86581222]
[ 0.53440199  0.2845992   0.09996637  0.26148603  0.78321856  2.91572563]
[ 0.53207403  0.28406674  0.10209586  0.26115933  0.77098581  2.82588813]
[ 0.52296193  0.29317535  0.10216163  0.25781885  0.76199365  2.83230377]
[ 0.51465883  0.29875601  0.10409886  0.25340565  0.74354355  2.75306613]
[ 0.08579111  0.63325167  0.18858545  0.00507613  0.37902409  1.70336902]
[ 0.5276873   0.28894158  0.10171867  0.25784228  0.76639271  2.84562598]
[ 0.52109764  0.29180148  0.10490933  0.25803268  0.74712637  2.76511435]
[ 0.51172514  0.29742622  0.10737626  0.25494477  0.72953365  2.6584236 ]
[ 0.51637433  0.29609456  0.10472817  0.25413579  0.74845069  2.72547711]
[ 0.52696448  0.2891813   0.10207797  0.25736503  0.7663825   2.82847699]
[ 0.51609501  0.29682976  0.10494238  0.2556042   0.74381303  2.76594894]
[ 0.52331802  0.29135904  0.10326355  

  ### Now let's create arrays of the fluctuations of the best fit parameters popti around the true solution popt0

In [15]:
omegas1 = np.array(omegas1) - w10
omegas2 = np.array(omegas2) - w20
omegas3 = np.array(omegas3) - w30
omegas4 = np.array(omegas4) - w40
sigmas1 = np.array(sigmas1) - s10
sigmas2 = np.array(sigmas2) - s20
sigmas3 = np.array(sigmas3) - s30

In [18]:
## now computing the standard deviation of the best-fit parameters population
stdw1, stdw2,stdw3,stdw4,stds1,stds2,stds3 = omegas1.std(), omegas2.std(), omegas3.std(),omegas4.std(), sigmas1.std(),sigmas2.std(),sigmas3.std()
print "Bootstrap errors in w1,w2,w3,w4,s1,s2,s3 are:", stdw1, stdw2,stdw3,stdw4,stds1,stds2,stds3

Bootstrap errors in w1,w2,w3,w4,s1,s2,s3 are: 0.0589445684659 0.0462391309024 0.0116198152562 0.00639317317352 0.114809979626 0.0675430178292 0.149844830343
